In [ ]:
import numpy as np
from gearbox_model import RxGearboxModel, TxGearboxModel
import matplotlib.pyplot as plt

## Tx Gearbox

In [ ]:

n_frames = 200
header = [[f'{y:02d}-H0{x}' for x in range(2)] for y in range(n_frames)]
data = [[f'{y:02d}-D{x:02d}' for x in range(64)] for y in range(n_frames)]


txg = TxGearboxModel('str')
txg_odata = []

print(txg.get_state_header())

for h, d in zip(header, data):
    for _ in range(2):
        idata = d[32:] if txg.get_frame_word() else d[:32]
        ret = txg.next(h, idata)
        txg_odata.append(ret['data'])
        print(txg.get_state())

        if (ret['pause']):
            ret = txg.next(h, idata)
            txg_odata.append(ret['data'])
            print(txg.get_state())

## Rx Gearbox - no slip

In [ ]:
rxg = RxGearboxModel('str')

rxg_oheader =[]
rxg_odata = []

for id in txg_odata:
    ret = rxg.next(id)
    print(rxg.get_state())

    if ret['header_valid']:
        rxg_oheader.append(ret['header'])

    if ret['data_valid']:
        rxg_odata.append(ret['data'])

In [ ]:
# check the data

# stitch the data back together in to frames



oframes = [list(np.concatenate([x,y])) for x,y in zip(rxg_odata[::2], rxg_odata[1::2])]


head_match = [ih == oh for ih, oh in zip(header, rxg_oheader)]
data_match = [id == od for id, od in zip(data, oframes)]

plt.plot(head_match)
plt.plot(data_match)

## Rx Gearbox - half slip needed

In [ ]:
# try with 1 slip out
rx_idata = txg_odata.copy()
rx_idata = [item for sublist in rx_idata for item in sublist]
for _ in range(7): rx_idata.insert(0, 'XXX')

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

rx_idata = list(chunks(rx_idata, 32))[:-1]

In [ ]:
rxg = RxGearboxModel('str')
slip_next = False

rxg_oheader = []
rxg_odata = []

for id in rx_idata:
    ret = rxg.next(id, slip_next)
    print(rxg.get_state())

    aligned = ('H00' in ret['header'][0] and 'H01' in ret['header'][1])

    slip_next = ret['header_valid'] and not aligned and not slip_next 

    if ret['header_valid']:
        rxg_oheader.append(ret['header'])

    if ret['data_valid']:
        rxg_odata.append(ret['data'])